## Imports

In [ ]:
import warnings

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from potentiel_solaire.database.queries import get_high_priority_schools
from potentiel_solaire.features.roof_attributes import segmentation_toits, recuperation_mns

from potentiel_solaire.constants import RESULTS_FOLDER

warnings.filterwarnings("ignore")

## Constants

In [ ]:
DEFAULT_CRS_METERS = 2154

## Bâtiments des écoles prioritaires pour Greenpeace

In [ ]:
priotirized_schools = get_high_priority_schools()

priotirized_schools_buildings = []
for code_departement in priotirized_schools["code_departement"].unique():
    # Read results for the current department
    path_results = RESULTS_FOLDER / f"D{code_departement}_pipeline_results.gpkg"
    schools_buildings_dep = gpd.read_file(
        path_results, layer="solar_potential_of_schools_buildings"
    ).to_crs(DEFAULT_CRS_METERS)[[
        "cleabs_bat", 
        "identifiant_de_l_etablissement", 
        "surface_totale_au_sol", 
        "surface_utile", 
        "geometry"
    ]]

    # Get priotirized schools for the current department
    priotirized_schools_dep = priotirized_schools[priotirized_schools["code_departement"] == code_departement]

    # Filter priotirized schools buildings for the current department
    priotirized_schools_buildings_dep = schools_buildings_dep.merge(
        priotirized_schools_dep[["identifiant_de_l_etablissement", "code_region", "code_departement"]],
        on="identifiant_de_l_etablissement",
        how="inner"
    )

    # Append to the list
    priotirized_schools_buildings.append(priotirized_schools_buildings_dep)


# Concatenate all departements geodataframes
priotirized_schools_buildings = gpd.GeoDataFrame(
    pd.concat(priotirized_schools_buildings, ignore_index=True), 
    crs=DEFAULT_CRS_METERS
)

nb_priotirized_schools = len(priotirized_schools_buildings["identifiant_de_l_etablissement"].unique())
nb_priotirized_schools_buildings = len(priotirized_schools_buildings)
print(f"Nombre d'établissements scolaires prioritaires : {nb_priotirized_schools}")
print(f"Nombre de bâtiments d'établissements scolaires prioritaires : {nb_priotirized_schools_buildings}")

In [ ]:
solar_potential_of_schools_buildings_sampled = priotirized_schools_buildings[priotirized_schools_buildings["code_departement"] == "063"].reset_index(drop=True).copy()
solar_potential_of_schools_buildings_sampled["index"] = solar_potential_of_schools_buildings_sampled.index

nb_buildings_sampled = len(solar_potential_of_schools_buildings_sampled)
print(f"Nombre de bâtiments d'établissements scolaires prioritaires échantillonnés : {nb_buildings_sampled}")

## Calculs en utilisant la segmentation des toits via les MNS

In [ ]:
def compute_roof_attributes_with_mns(building):
    print(building["index"], "/", nb_buildings_sampled)
          
    building = gpd.GeoDataFrame(
        [building.values], 
        columns=solar_potential_of_schools_buildings_sampled.columns, 
        crs=DEFAULT_CRS_METERS
    ).to_crs(2154)

    result = segmentation_toits(recuperation_mns(building), min_surface=2)

    surface_utile = result["surface"].sum()

    return pd.Series({"mns_surface_utile": surface_utile})

solar_potential_of_schools_buildings_sampled["mns_surface_utile"] = solar_potential_of_schools_buildings_sampled.apply(
    compute_roof_attributes_with_mns,
    axis=1,
)

## Comparaison de la surface utile calculée par le MNS avec la notre

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]

surface_au_sol = solar_potential_of_schools_buildings_sampled["surface_totale_au_sol"]
surface_utile = solar_potential_of_schools_buildings_sampled["surface_utile"]
surface_utile_mns = solar_potential_of_schools_buildings_sampled["mns_surface_utile"]

# plot identity line
identity_line = [0, surface_utile.max()]
plt.plot(identity_line, identity_line, "r--")


# compare with results using MNS
plt.scatter(
    surface_utile,
    surface_utile_mns,
    color="green",
    alpha=0.8,
)

plt.xlabel("Surface utile (methode simplifiée)")
plt.ylabel("Surface utile (segmentation des toits avec MNS)")
plt.title("Methode simplifiée vs methode de segmentation des toits avec MNS")

plt.show()

In [ ]:
solar_potential_of_schools_buildings_sampled["euclidean_distance"] = abs(solar_potential_of_schools_buildings_sampled["surface_utile"] - solar_potential_of_schools_buildings_sampled["mns_surface_utile"])
solar_potential_of_schools_buildings_sampled["euclidean_distance"].describe()

## Verification si la surface au sol du batiment reste un facteur determinant

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]

surface_au_sol = solar_potential_of_schools_buildings_sampled["surface_totale_au_sol"]
surface_utile = solar_potential_of_schools_buildings_sampled["surface_utile"]
surface_utile_mns = solar_potential_of_schools_buildings_sampled["mns_surface_utile"]

# plot max possible surface utile
max_surface_utile = [0, surface_au_sol.max()]
plt.plot(max_surface_utile, max_surface_utile, "r--")


# results with simplified method
plt.scatter(
    surface_au_sol,
    surface_utile,
    color="blue",
    alpha=0.8,
)


# compare with results using MNS
plt.scatter(
    surface_au_sol,
    surface_utile_mns,
    color="green",
    alpha=0.8,
)

plt.xlabel("Surface totale au sol (m²)")
plt.ylabel("Surface utile (m²)")
plt.title("Methode simplifiée vs methode de segmentation des toits avec MNS")

plt.show()